In [1]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://52.6.70.195:7687",
  auth=basic_auth("neo4j", "rank-composition-bomb"))
session = driver.session()

In [2]:
cypher_query = '''
Match (n)
DETACH DELETE n
'''
results = session.run(cypher_query)

In [3]:
# Make Figure 1
cypher_query = '''
//Patient
create (peter:Subject:User:Primitive
{name:'Peter'})-[:HAS_ATTR]->(patient:Role:Attribute {name:'Patient'})

//Doctor & Nurses
create (joe:Subject:User:Primitive {name:"Joe"})-[:HAS_ATTR]->(nurse:Role:Attribute {name:"Nurse"})
create (john:Subject:User:Primitive {name:'John'})-[:HAS_ATTR]->(doctor:Role:Attribute {name:"Doctor"})
merge (john)-[:HAS_ATTR]->(staff:Group:Attribute {name:'Hospital Staff'})<-[:HAS_ATTR]-(joe)
merge (sue:Subject:User:Primitive {name: "Sue"})-[:HAS_ATTR]->(doctor)
merge (sue)-[:HAS_ATTR]->(pClinic:Relationship:Attribute {name: "Peter's Family Clinic"})

//Peter's digital assets
merge (peter)-[:OWNER_OF]->(pRecs:Group:Attribute {name: "Peter's Medical Records"})<-[:HAS_ATTR]-(rec:Record:Object:Primitive {name:"MR_1234"})
merge (pRecs)-[:HAS_ATTR]->(hRecs:Group:Attribute {name:'Hospital Records'})
merge (peter)-[:OWNER_OF]->(prof:Data:Object {name:"Peter's Profile"})-[:HAS_ATTR]->(pp:Group:Attribute {name:'Hospital Profiles'})

//Actions
merge (read:Action {name:'Read'})-[:HAS_ATTR]->(fullAccess:Attribute:Group {name:"Full Access"})<-[:HAS_ATTR]-(write:Action {name:'Write'});
'''
results = session.run(cypher_query)

In [4]:
# Make Figure 2
cypher_query = '''
// Policy 1 - Permit; Hospital Staff, Full Access, Patient Profile
match (sub:Attribute {name:"Hospital Staff"}), (obj:Attribute 
{name:"Hospital Profiles"}), (act:Attribute {name:"Full Access"})
create (pol:Policy {name:'Policy1', decision:"Permit"})
merge (pol)<-[:SUB_CON]-(sub)
merge (pol)<-[:OBJ_CON]-(obj)
merge (pol)<-[:ACT_CON]-(act);
'''
results = session.run(cypher_query)

In [5]:
# Making Figure 2 cont..

cypher_query = '''
// Policy 2 - Permit: Doctor & Hospital Staff, Full Access, Hospital Records
match (sub1: Attribute {name:"Hospital Staff"}), (sub2:Attribute 
{name:"Doctor"}), (obj:Attribute {name:"Hospital Records"}), (act:Attribute {name:"Full Access"})
create (pol:Policy {name:"Policy2", decision:"Permit"})
merge (pol)<-[:SUB_CON]-(sub1)
merge (pol)<-[:SUB_CON]-(sub2)
merge (pol)<-[:OBJ_CON]-(obj)
merge (pol)<-[:ACT_CON]-(act);
'''
results = session.run(cypher_query)

In [6]:
# Making Figure 2 cont..

cypher_query = '''
// Policy 3 - Permit: Doctor & Peter's Family Clinic, Read, Peter's Medical Records
match (sub2:Attribute {name:"Doctor"}), (act:Action {name:"Read"}), (obj:Attribute {name:"Peter's Medical Records"}), (sub1:Attribute {name:"Peter's Family Clinic"})
create (pol:Policy {name:"Policy3", decision:"Permit"})
merge (pol)<-[:OBJ_CON]-(obj)
merge (pol)<-[:SUB_CON]-(sub2)
merge (pol)<-[:SUB_CON]-(sub1)
merge (pol)<-[:ACT_CON]-(act);
'''
results = session.run(cypher_query)

In [28]:
cypher_query = '''
match (sub)-[:HAS_ATTR*0..10]->(sc)-[:SUB_CON]->(pol:Policy)
with pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:SUB_CON]-(rc)
with pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons
match (obj)-[:HAS_ATTR*0..2]->(sc)-[:OBJ_CON]->(pol)
with pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:OBJ_CON]-(rc)
with pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons
match (act)-[:HAS_ATTR*0..10]->(sc)-[:ACT_CON]->(pol) with pol, size(collect(distinct sc)) as sat_cons match (pol)<-[:ACT_CON]-(rc)
with pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons
return pol as matching_policies

'''
results = session.run(cypher_query)

In [ ]:
for line in results:
    print(line)

## From here on this doesn't work

In [23]:
# Why this doesn't work

cypher_query = '''
:params {"JohnWriteMR_1234": {"SUBJECT_NAME":"John","OBJECT_NAME": "MR_1234", "ACTION_NAME":"Write"}, "SueReadMR_1234": {"SUBJECT_NAME":"Sue", "OBJECT_NAME":"MR_1234", "ACTION_NAME":"Read"}, "SueWriteMR_1234": {"SUBJECT_NAME": "Sue", "OBJECT_NAME": "MR_1234", "ACTION_NAME": "Write"}}
'''
results = session.run(cypher_query)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ':': expected <init> (line 2, column 1 (offset: 1))
":params {"JohnWriteMR_1234": {"SUBJECT_NAME":"John","OBJECT_NAME": "MR_1234", "ACTION_NAME":"Write"}, "SueReadMR_1234": {"SUBJECT_NAME":"Sue", "OBJECT_NAME":"MR_1234", "ACTION_NAME":"Read"}, "SueWriteMR_1234": {"SUBJECT_NAME": "Sue", "OBJECT_NAME": "MR_1234", "ACTION_NAME": "Write"}}"
 ^}

In [26]:
# This is the policy search

cypher_query = '''
:params {"JohnWriteMR_1234": {"SUBJECT_NAME": "John", "OBJECT_NAME": "MR_1234", "ACTION_NAME": "Write"}}

with $JohnWriteMR_1234 as req
// Stage 1 - Subject Conditions
match (sub:Subject {name:req.SUBJECT_NAME})-[:HAS_ATTR*0..5]->(sc)-[:SUB_CON]->(pol:Policy)
with req, pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:SUB_CON]-(rc)
with req, pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons 

// Stage 2 - Object Conditions
match (obj:Object {name:req.OBJECT_NAME})-[:HAS_ATTR*0..5]->(sc)-[:OBJ_CON]->(pol) 
with req, pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:OBJ_CON]-(rc)
with req, pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons 

// Stage 3 - Action Conditions
match (act:Action {name:req.ACTION_NAME})-[:HAS_ATTR*0..5]->(sc)-[:ACT_CON]->(pol) 
with req, pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:ACT_CON]-(rc)
with req, pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons 
return case when count(pol) = 0 or 'Deny' in collect(pol.decision) then 'Deny' else 'Permit' end as decision
'''
results = session.run(cypher_query)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ':': expected <init> (line 2, column 1 (offset: 1))
":params {"JohnWriteMR_1234": {"SUBJECT_NAME": "John", "OBJECT_NAME": "MR_1234", "ACTION_NAME": "Write"}}"
 ^}

In [ ]:
for line in results:
    print(line)

In [27]:
# Testing

graph_db = neo4j.
qs = '''
with {JohnWriteMR_1234} as req
// Stage 1 - Subject Conditions
match (sub:Subject {name:req.SUBJECT_NAME})-[:HAS_ATTR*0..5]->(sc)-[:SUB_CON]->(pol:Policy)
with req, pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:SUB_CON]-(rc)
with req, pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons 

// Stage 2 - Object Conditions
match (obj:Object {name:req.OBJECT_NAME})-[:HAS_ATTR*0..5]->(sc)-[:OBJ_CON]->(pol) 
with req, pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:OBJ_CON]-(rc)
with req, pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons 

// Stage 3 - Action Conditions
match (act:Action {name:req.ACTION_NAME})-[:HAS_ATTR*0..5]->(sc)-[:ACT_CON]->(pol) 
with req, pol, size(collect(distinct sc)) as sat_cons
match (pol)<-[:ACT_CON]-(rc)
with req, pol, sat_cons, size(collect(rc)) as req_cons where req_cons = sat_cons 
return case when count(pol) = 0 or 'Deny' in collect(pol.decision) then 'Deny' else 'Permit' end as decision
'''

SyntaxError: invalid syntax (<ipython-input-27-7731d00f060b>, line 1)